# Sentinel 2 Atmospheric Correction in Google Earth Engine

### Import modules 
and initialize Earth Engine

In [1]:
import ee
from Py6S import *
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric
from AtcorFunctions import *

ee.Initialize()

### time and place
Define the time and place that you are looking for.

In [2]:
# start and end of time series
start = ee.Date('2018-08-01')
finish =ee.Date('2018-08-31')

#coordinates below need adjusting to actual site coordinates - these are just close
geom = ee.Geometry.Point([8.1191,60.0676]) 
aoi = geom.buffer(10000).bounds() #buffer is in meters, can adjust (10km buffer here)

# Whole park
#geom = ee.Geometry.Rectangle(8.3018, 60.3967,6.7596,59.9)

### an image
The following code will grab the first scene that occurs on or after date.

In [3]:
collection = ee.ImageCollection("COPERNICUS/S2")\
    .filterBounds(geom)\
    .filterDate(start,finish)\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 80)
    #.map(s2mask())

# The first image in the collection
S2 = ee.Image(collection.first())

print(ee.Date(S2.get('system:time_start')).format('yyyy-M-d').getInfo())
print(ee.Date(S2.get('system:time_start')).format('yyyy-mm-dd').getInfo())

# top of atmosphere reflectance
toa = S2.divide(10000)

2018-8-4
2018-50-04


In [4]:
date=start
## METADATA
info = S2.getInfo()['properties']
scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)# i.e. Python uses seconds, EE uses milliseconds
solar_z = info['MEAN_SOLAR_ZENITH_ANGLE']

## ATMOSPHERIC CONSTITUENTS
h2o = Atmospheric.water(geom,date).getInfo()
o3 = Atmospheric.ozone(geom,date).getInfo()
aot = Atmospheric.aerosol(geom,date).getInfo()

## TARGET ALTITUDE
SRTM = ee.Image('USGS/GMTED2010')# Shuttle Radar Topography mission covers *most* of the Earth
alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('be75').getInfo()
km = alt/1000 # i.e. Py6S uses units of kilometers

### 6S object

The backbone of Py6S is the 6S (i.e. SixS) class. It allows you to define the various input parameters, to run the radiative transfer code and to access the outputs which are required to convert radiance to surface reflectance.

In [5]:
# Instantiate
s = SixS()

# Atmospheric constituents
s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
s.aero_profile = AeroProfile.Continental
s.aot550 = aot

# Earth-Sun-satellite geometry
s.geometry = Geometry.User()
s.geometry.view_z = 0               # always NADIR (I think..)
s.geometry.solar_z = solar_z        # solar zenith angle
s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
s.altitudes.set_sensor_satellite_level()
s.altitudes.set_target_custom_altitude(km)

### Atmospheric Correction

In [9]:
def spectralResponseFunction(bandname):
    """
    Extract spectral response function for given band name
    """
    bandSelect = {
        'B1':PredefinedWavelengths.S2A_MSI_01,
        'B2':PredefinedWavelengths.S2A_MSI_02,
        'B3':PredefinedWavelengths.S2A_MSI_03,
        'B4':PredefinedWavelengths.S2A_MSI_04,
        'B5':PredefinedWavelengths.S2A_MSI_05,
        'B6':PredefinedWavelengths.S2A_MSI_06,
        'B7':PredefinedWavelengths.S2A_MSI_07,
        'B8':PredefinedWavelengths.S2A_MSI_08,
        'B8A':PredefinedWavelengths.S2A_MSI_09,
        'B9':PredefinedWavelengths.S2A_MSI_10,
        'B10':PredefinedWavelengths.S2A_MSI_11,
        'B11':PredefinedWavelengths.S2A_MSI_12,
        'B12':PredefinedWavelengths.S2A_MSI_13,
        }
    return Wavelength(bandSelect[bandname])
def toa_to_rad(bandname):
    """
    Converts top of atmosphere reflectance to at-sensor radiance
    """
    # solar exoatmospheric spectral irradiance
    ESUN = info['SOLAR_IRRADIANCE_'+bandname]
    solar_angle_correction = math.cos(math.radians(solar_z))
    # Earth-Sun distance (from day of year)
    doy = scene_date.timetuple().tm_yday
    d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
    # conversion factor
    multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
    # at-sensor radiance
    rad = toa.select(bandname).multiply(multiplier)
    return rad
def surface_reflectance(bandname):
    """
    Calculate surface reflectance from at-sensor radiance given waveband name
    """
    # run 6S for this waveband
    s.wavelength = spectralResponseFunction(bandname)
    s.run()
    # extract 6S outputs
    Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
    Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
    Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
    absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
    scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
    tau2 = absorb*scatter                                #total transmissivity
    # radiance to surface reflectance
    rad = toa_to_rad(bandname)
    ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
    return ref
# surface reflectance rgb
b = surface_reflectance('B2')
g = surface_reflectance('B3')
r = surface_reflectance('B4')
ref = r.addBands(g).addBands(b)

# # all wavebands
output = S2.select('QA60')
for band in ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12']:
    print(band)
    output = output.addBands(surface_reflectance(band))

B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12


### Display results

In [10]:
from IPython.display import display, Image

region = geom.buffer(10000).bounds().getInfo()['coordinates']
channels = ['B4','B3','B2']

original = Image(url=toa.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25
                }))

corrected = Image(url=output.select(channels).getThumbUrl({
                'region':region,'min':0,'max':0.25
                }))

display(original, corrected)

## Cloud mask functions (from *geetools*)

In [11]:
# Import relevant functions
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'geetools'))
from geetools import ui, cloud_mask

In [12]:
MapS2 = ui.Map(tabs=('Inspector',))
MapS2.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
#visS2 = {'bands':['B4','B3','B2'],'min':0, 'max':5000}
visS2 = {min: 0.0,max: 0.25,'bands':channels}
is2=output#S2/output
is2=is2.clip(aoi)

MapS2.centerObject(is2, zoom=11)
MapS2.addLayer(is2,visS2, 'Sentinel 2 Original')

### ESA Cloud mask

In [14]:
ESA_mask_all = cloud_mask.sentinel2()
is2_ESA = ESA_mask_all(is2)
MapS2.addLayer(is2_ESA, visS2, 'Sentinel 2 ESA maked')

### Sentinel Hub Cloud mask (*uses machine learning*)

In [15]:
import s2cloudless
s2cloudless.test_sentinelhub_cloud_detector() 

INFO:s2cloudless.test_cloud_detector:Test OK.
Cloud probabilities and cloud masks match templates.


### Export to Asset

In [16]:
# # set some properties for export
dateString = scene_date.strftime("%Y-%m-%d")
ref = ref.set({'satellite':'Sentinel 2',
              'fileID':info['system:index'],
              'date':dateString,
              'aerosol_optical_thickness':aot,
              'water_vapour':h2o,
              'ozone':o3})

In [17]:
# define YOUR assetID 
assetID = 'users/visithuruvixen/test'

In [18]:
# # export
export = ee.batch.Export.image.toAsset(\
    image=output,
    description='sentinel2_atmcorr_export',
    assetId = assetID,
    region = region,
    scale = 30)

# # uncomment to run the export
export.start() 